In [1]:
from symtable import Symbol
from StockStageEstimator import StockStageEstimator
import pandas as pd
import numpy as np
import random
from datetime import datetime
from stage import fullPrint
# import mariadb
import sys
import os
# from dotenv import load_dotenv

# load_dotenv()

### ENV CONSTANT
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')

### Connect To DB
# try:
#     conn = mariadb.connect(
#         user=DB_USER,
#         password=DB_PASSWORD,
#         host=DB_HOST,
#         port=3306,
#         database="stock",
#         autocommit=True
#     )
# except mariadb.Error as e:
#     print(f"Error connecting to MariaDB Platform: {e}")
#     sys.exit(1)
# cur = conn.cursor()

### CONSTANT VARIABLES ###

## Mutation Noise 
mutationNoises = []
mutationNoises.append(np.random.normal(0, 0.025)) # 0 - Five Year High
mutationNoises.append(np.random.normal(0, 0.075)) # 1 - Volume Perc
mutationNoises.append(np.random.normal(0, 0.015)) # 2 - RS
mutationNoises.append(np.random.normal(0, 0.005)) # 3 - Slope
mutationNoises.append(np.random.normal(0, 0.0125)) # 4 - Price v WMA30
mutationNoises.append(np.random.normal(0, 0.015)) # 5 - Price v Initial Support
mutationNoises.append(np.random.normal(0, 0.005)) # 6 - Prev Trough v Peak
mutationNoises.append(np.random.normal(0, 0.01)) # 7 - Price v Prev Support
mutationNoises.append(np.random.normal(0, 0.0125)) # 8 - Price v Prev Close
mutationNoises.append(np.random.normal(0, 0.0005)) # 9 - Sector MA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 10 - Sector RS
mutationNoises.append(np.random.normal(0, 0.0025)) # 11 - SPDF Price v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 12 - SPDF WMA30 Slope
mutationNoises.append(np.random.normal(0, 0.0025)) # 13 - SPDF Sell P v WMA30
mutationNoises.append(np.random.normal(0, 0.0002)) # 14 - SPDF Sell WMA30 Slope


In [2]:
## Get returns of a ticker list
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for industry in list:
        symbol = industry[1]
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/industriesData/"+industry[0]+'/'+symbol+".pkl")
            print(df)
        except:
            print(symbol)
            continue
        if not (df['Close'] != 0).all() or df.empty:
            if df.empty:
                print(symbol)
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = monthlyReturn.index[0]
        lastPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            if (index.strftime("%Y:%m") != monthCheck):
                if (initialPrice == 0): continue
                if (lastPrice / initialPrice >= 1.01 and lastPrice / initialPrice < 2):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (lastPrice / initialPrice)
                initialPrice = element[0]
                lastPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            lastPrice = element[0]
    return monthlyReturn

In [3]:
print(pd.read_pickle("transactionTemplate.pkl"))

Empty DataFrame
Columns: []
Index: [2010-01-03 00:00:00, 2010-01-10 00:00:00, 2010-01-17 00:00:00, 2010-01-24 00:00:00, 2010-01-31 00:00:00, 2010-02-07 00:00:00, 2010-02-14 00:00:00, 2010-02-21 00:00:00, 2010-02-28 00:00:00, 2010-03-07 00:00:00, 2010-03-14 00:00:00, 2010-03-21 00:00:00, 2010-03-28 00:00:00, 2010-04-04 00:00:00, 2010-04-11 00:00:00, 2010-04-18 00:00:00, 2010-04-25 00:00:00, 2010-05-02 00:00:00, 2010-05-09 00:00:00, 2010-05-16 00:00:00, 2010-05-23 00:00:00, 2010-05-30 00:00:00, 2010-06-06 00:00:00, 2010-06-13 00:00:00, 2010-06-20 00:00:00, 2010-06-27 00:00:00, 2010-07-04 00:00:00, 2010-07-11 00:00:00, 2010-07-18 00:00:00, 2010-07-25 00:00:00, 2010-08-01 00:00:00, 2010-08-08 00:00:00, 2010-08-15 00:00:00, 2010-08-22 00:00:00, 2010-08-29 00:00:00, 2010-09-05 00:00:00, 2010-09-12 00:00:00, 2010-09-19 00:00:00, 2010-09-26 00:00:00, 2010-10-03 00:00:00, 2010-10-10 00:00:00, 2010-10-17 00:00:00, 2010-10-24 00:00:00, 2010-10-31 00:00:00, 2010-11-07 00:00:00, 2010-11-14 00:00:00

In [4]:
industryList = pd.read_pickle("stockData/industryList.pkl")
calculateGroupReturn(industryList)

             Close     Volume  fiveYearHigh       WMA30  WMA30Slope  \
Date                                                                  
2010-01-03  313.78   41430400           0.0         NaN    0.000000   
2010-01-10  316.48   37572100           0.0         NaN    0.000000   
2010-01-17  301.59   43442500           0.0         NaN    0.000000   
2010-01-24  297.69   27905700           0.0         NaN    0.000000   
2010-01-31  295.59   33383700           0.0         NaN    0.000000   
...            ...        ...           ...         ...         ...   
2021-11-28  661.16   77387504         992.9  712.886688    0.994317   
2021-12-05  670.72   59063500         992.9  709.581935    0.995364   
2021-12-12  704.16  100599104         992.9  708.573720    0.998579   
2021-12-19  693.16   50983900         992.9  706.916882    0.997662   
2021-12-26  706.67   39817500         992.9  706.197097    0.998982   

            volumePerc   percent        RS    peak  trough  support  \
Date 

/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12143/1201086260.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12143/1201086260.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  monthlyReturn[symbol] = 1
/var/folders/1q/w3g5nkks29g3kjdwmq1lssj00000gn/T/ipykernel_12143/1201086260.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performa

,DJTobacco,DJSoftDrinks,DJDrugRetailers,DJDistillers&Vintners,DJFoodProductMakers,DJFoodRetail&Wholesale,DJNondurableHousehold,DJBrewers,DJPersonalProducts,DJGasDistribution,...,DJRestaurants&Bars,DJHomeImprovementRetailers,DJFurnishings,DJSpecializedConsumerServices,DJHotels,DJRecreationalProducts,DJGambling,DJDurableHouseholdProducts,DJToys,DJApparelRetailers
Date,,,,,,,,,,,,,,,,,,,,,
2010:01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2010:02,1.000000,1.000000,1.000000,1.000000,1.000000,1.015742,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2010:03,1.041897,1.046152,1.056103,1.068579,1.050980,1.079926,1.032925,1.100512,1.048751,1.074285,...,1.019557,1.091969,1.128327,1.084821,1.069675,1.078106,1.020870,1.081022,1.045886,1.118362
2010:04,1.038351,1.030192,1.066938,1.042325,1.017186,1.012273,1.000000,1.000000,1.031771,1.013643,...,1.026780,1.000000,1.000000,1.020110,1.095276,1.000000,1.077789,1.000000,1.021536,1.035078
2010:05,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.017883,1.000000,1.010719,...,1.024062,1.059312,1.128047,1.000000,1.066643,1.047327,1.051205,1.063936,1.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021:08,1.000000,1.046396,1.022069,1.000000,1.000000,1.031982,1.032800,1.000000,1.000000,1.000000,...,1.048722,1.010690,1.000000,1.051941,1.010359,1.015122,1.000000,1.000000,1.000000,1.000000
2021:09,1.071971,1.010186,1.066848,1.000000,1.000000,1.060018,1.019229,1.000000,1.028120,1.000000,...,1.000000,1.024412,1.000000,1.000000,1.000000,1.000000,1.086706,1.000000,1.000000,1.000000
2021:10,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.123444,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
## Calculate Goodsector & Initialize Estimator Object 
def evalReturn(individual):
    now = datetime.now()
    now = now.strftime("%Y-%m-%d-%H:%M")
    transaction = pd.read_pickle("transactionTemplate.pkl")
    transaction['holding'] = np.empty((len(transaction), 0)).tolist()
    SectorDict = {
    "Energy":"XLE","Technology":"XLK",
    "Communication Services":"XLC",
    "Consumer Staples":"XLP",
    "Health Care":"XLV",
    "Consumer Cyclical":"XLY",
    "Industrial":"XLI",
    "Utilities":"XLU",
    "Materials":"XLB",
    "Financial":"XLF",
    "Real Estate":"XLRE"}
    sectorDfList = []
    for sector in SectorDict.values():
        sectorDfList.append(pd.read_pickle("stockData/sectorCharts/"+sector+".pkl"))
        goodSectorDf = sectorDfList[0].drop(['open','high','low','adjclose','close','ticker','volume','percent','MA30','MA30Slope','RS'],axis=1)
        goodSectorDf['Sectors'] = [[] for _ in range(len(goodSectorDf))]
        goodSectorDf.index = sectorDfList[0].index
        index = 0
        for date in sectorDfList[0].index:
            listOfSector = []
            for df in sectorDfList:
                try:
                    # print(df.at[date,'MA30Slope'],df.at[date,'RS'])
                    # print(individual[0][9],individual[0][10])
                    if  df.at[date,'MA30Slope'] > individual[0][9] and df.at[date,'RS'] > individual[0][10]:
                        if df.at[date,'close'] > df.at[date,'MA30']:
                            # print(df.at[date,'ticker'])
                            listOfSector.append(df.at[date,'ticker'])
                except: 
                    pass
                # print(listOfSector)
            goodSectorDf.iat[index,0] = set(listOfSector)
            index = index + 1
    stockStageEstimator = StockStageEstimator(individual[0], goodSectorDf)
    scores = stockStageEstimator.score()
    debugFile = open("estimatorData/debug"+date.today().strftime('%Y-%m-%d')+".txt","a")
    debugFile.write(str(individual[0])+"\n")
    if scores == -1:
        debugFile.write(str(np.average(scores))+"\n")
        debugFile.write(str(stockStageEstimator.getReturns())+"\n")
        debugFile.close()
        return 0,
    result = stockStageEstimator.result()
    debugFile.write(str(np.average(scores[0]))+"\n")
    debugFile.write(str(np.average(scores[1]))+"\n")
    debugFile.write(str(stockStageEstimator.getReturns())+"\n")
    debugFile.write(str(result)+"\n")
    debugFile.close()
    if np.average(scores[0]) >  0.1:
        return 0,
    if np.average(scores[1]) > 1.3:
        return 0,
    paramFile = open("estimatorData/params"+date.today().strftime('%Y-%m-%d')+".txt","a")
    paramFile.write(str(individual[0])+"\n")
    paramFile.write(str(result)+"\n")
    paramFile.close()

In [ ]:
    # cur.execute("INSERT INTO Params (param,result) VALUES (?, ?)", (individual[0], result))

    resultFile = open("estimatorData/resultML"+date.today().strftime('%Y-%m-%d')+".txt","a")
    resultFile.write(str(individual[0])+"\n")
    resultFile.write(str(np.average(scores[0]))+"\n")
    resultFile.write(str(np.average(scores[1]))+"\n")
    resultFile.write(str(stockStageEstimator.getReturns())+"\n")
    resultFile.write(str(result)+"\n")
    resultFile.close()
    return result,


In [ ]:
### DEAP Evolutionary Functions ###

## Generating individuals
def generate_random_num_attr():
  original = np.array([0,  # 0 - Five Year High
                        0, # 1 - Volume Perc
                        0, # 2 - RS
                        0, # 3 - Slope
                        0, # 4 - Price v WMA30
                        0, # 5 - Price v Initial Support
                        0, # 6 - Prev Trough v Peak
                        0, # 7 - Price v Prev Support
                        0, # 8 - Price v Prev Close
                        0.995, # 9 - Sector MA30 Slope
                        0, # 10 - Sector RS
                        0.9, # 11 - SPDF Price v WMA30
                        0.995, # 12 - SPDF WMA30 Slope
                        0.85, # 13 - SPDF Sell P v WMA30
                        0.985]) # 14 - SPDF Sell WMA30 Slope
  noises = []
  sum_list = []

In [ ]:
  noises.append(random.uniform(0,1))
  noises.append(random.uniform(0.1,2.5))
  noises.append(random.uniform(-0.1,0.4))
  noises.append(random.uniform(0.9,1.1))
  noises.append(random.uniform(1,1.4))
  noises.append(random.uniform(1, 1.3))
  noises.append(random.uniform(0.8, 1))
  noises.append(random.uniform(0.8, 1.1))
  noises.append(random.uniform(1.2, 1.5))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.2))

  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))
  noises.append(np.random.normal(0, 0.05))
  noises.append(np.random.normal(0, 0.008))

  while original[11] + noises[11] < original[13] + noises[13]:
    noises[11] = np.random.normal(0, 0.05)
    noises[13] = np.random.normal(0, 0.05)
  while original[12] + noises[12] < original[14] + noises[14]:
    noises[12] = np.random.normal(0, 0.004)
    noises[14] = np.random.normal(0, 0.004)
  
  for (item1, item2) in zip(original, noises):
      sum_list.append(item1+item2)
  
  return sum_list

In [ ]:
## Mutating Individuals
def mutate(individual):
    size = len(individual)
    for i in range(0,size):
        if random.random() > 0.9:
            individual[i] = individual[i] + mutationNoises[i]
    return individual,